In [1]:
import numpy as np
import torch, pickle
import lightgbm as lgb
from hummingbird import convert_sklearn
from onnxconverter_common.data_types import FloatTensorType

In [2]:
# load the LGBM model for Fraud
model = pickle.load(open('lgbm-500-6.pkl', "rb"))
X = np.random.rand(200000, 28)
X = np.array(X, dtype=np.float32)

pytorch_model = convert_sklearn(
    model, 
    [("input", FloatTensorType([200000, 28]))], 
    extra_config = {"tree_implementation": "perf_tree_trav"})

X_torch = torch.from_numpy(X)

In [3]:
%%timeit -r 3

#time for skl
skl = model.predict(X)

1.86 s ± 24.2 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [4]:
%%timeit -r 3

# time for hummingbird - CPU
pytorch_model.to('cpu')
hum_cpu = pytorch_model(X_torch)

5.72 s ± 473 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [5]:
%%timeit -r 3

# time for hummingbird - GPU. Note that you must have a GPU-enabled machine.
pytorch_model.to('cuda')
hum_gpu = pytorch_model(X_torch.to('cuda'))

163 ms ± 2.8 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [6]:
# make sure Hummingbird output matches Scikit-learn as expected
skl = model.predict_proba(X)
pytorch_model.to('cuda')
hum_gpu = pytorch_model(X_torch.to('cuda'))

np.testing.assert_allclose(skl, hum_gpu[1].data.to('cpu').numpy(), rtol=1e-06, atol=1e-06)